# Résolution de labyrinthes

Dans ce sujet de TP, sur 4 séances, nous allons voir comment implémenter la formalisation d'un problème de résolution par l'exploration ainsi que les différents algorithmes de recherche vus en cours puis en TD. Nous verrons enfin comment utiliser cela pour effectuer la résolution de labyrinthes.


In [213]:
from numpy.random import randint
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

# Définition d'un problème

On commence par définir une classe abstraite pour représenter un `Problem` qui sera résolu par l'exploration.

On indique l'état initial et soit l'état final (s'il y en a un seul), soit la manière de savoir si on se trouve dans un état final (en redéfinissant la méthode `is_goal`, dans la sous-classe qui implémente la classe `Problem`). La sous-classe devra également implémenter le constructeur `__init__`(en ajoutant éventuellement d'autres paramètres) et les méthodes `actions`, `result` et `action_cost`. 


In [214]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other arguments for the subclass."""


    def __init__(self, initial=None, goal=None): 
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal
        
    def actions(self, state):     
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""   
        raise NotImplementedError
    
    def result(self, state, action): 
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError
    
    def is_goal(self, state):   
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor."""     
        return state == self.goal
    
    def action_cost(self, s, a, s1): 
        """Return the cost of an action a from state s to state s1."""
        return 1
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

# Définition d'un noeud

Dans la classe `Node`, on définit un noeud dans un arbre de recherche.
Le noeud contient un pointeur vers son noeud parent `parent` ainsi que vers l'état `state` auquel il correspond (la notion d'état sera défini au moment de la définition du problème représenté). Il est à noter que si un état peut être atteint par 2 chemins, il y aura alors 2 noeuds contenant le même état. Le noeud contient également l'action `action` qui a permis d'atteindre l'état associé au noeud ainsi que le coût total `path_cost` du chemin allant du noeud initial au noeud courant (correspondant à la valeur g dans les algorithmes d'exploration).

In [215]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __repr__(self): 
        return '<{}>'.format(self.state)
    
    def __len__(self): 
        return 0 if self.parent is None else (1 + len(self.parent))
    
    def __lt__(self, other): 
        return self.path_cost < other.path_cost
    

## Fonctions relatives aux noeuds d'un arbre de recherche

On définit également quelques fonctions sur les noeuds : `expand` pour générer les noeuds successeurs d'un noeud donné, `path_actions` pour récupérer la liste d'actions ayant permis d'arriver jusqu'au noeud et `path_states` pour récupérer la liste des états ayant permis d'arriver jusqu'au noeud.

### Question 1 : compléter ces 3 fonctions

In [216]:
def expand(problem: Problem, node: Node): # Expand a node, generating the children nodes (see slide 18 in the lecture).
    ret = []

    for act in problem.actions(node.state):
        res = problem.result(node.state, act)
        
        ret.append(Node(
            res,
            node,
            act,
            node.path_cost + problem.action_cost(node.state, act, res)
        ))
    
    return ret


def path_actions(node: Node | None):
    if node.parent == None:
        return []
    
    return path_actions(node.parent) + [node.action]


def path_states(node: Node | None):
    if node == None:
        return []
    
    return path_states(node.parent) + [node.state]
    

# Différents types de files pour implémenter la frontière

Les files FIFO et LIFO sont utilisées, respectivement, pour implémenter la `frontière` dans l'exploration en largeur et dans l'exploration en profondeur. La file de priorié `PriorityQueue` est utilisée dans l'exploration gloutonne et dans l'exploration A* pour garder la frontière triée en fonction du score croissant `f(item)`.

In [217]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queue."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): 
        return self.items[0][1]

    def __len__(self): 
        return len(self.items)


# Algorithmes de recherche

Les algorithmes de recherche non-informée (exploration en largeur et exploration en profondeur, en mémorisant ou non les états déjà exploras) ainsi que les algorithmes de recherche informée (exploration gloutonne et A*) seront implémentés ici.

In [218]:
# Noeud particulier pour indiquer qu'un algorithme ne peut pas trouver de solution    
failure = Node('failure', path_cost=math.inf)

## Exploration en largeur de l'arbre de recherche

### Question 2 : implémenter l'exploration en largeur de l'arbre de recherche (voir slides 18 et 20 du cours)

In [219]:
def breadth_first_tree_search(problem: Problem): # Search the shallowest nodes in the search tree first.
    frontier = deque()

    frontier.append(Node(problem.initial))

    while len(frontier) > 0:
        currentNode: Node = frontier.popleft()

        if problem.is_goal(currentNode.state):
            return currentNode
        
        frontier.extend(expand(problem, currentNode))

    return None

## Exploration en profondeur de l'arbre de recherche

### Question 3 : implémenter l'exploration en profondeur de l'arbre de recherche (voir slides 18 et 21 du cours)

In [220]:
def depth_first_tree_search(problem: Problem):
    frontier = []

    frontier.append(Node(problem.initial))

    while len(frontier) > 0:
        currentNode: Node = frontier.pop()

        if problem.is_goal(currentNode.state):
            return currentNode
        
        frontier.extend(expand(problem, currentNode))

    return None

## Exploration en largeur du graphe des états

### Question 4 : implémenter l'exploration en largeur du graphe des états (voir slide 27 du cours)

In [221]:
def breadth_first_graph_search(problem: Problem):
    frontier = deque()
    explored: dict[tuple, bool] = {}

    frontier.append(Node(problem.initial))

    while len(frontier) > 0:
        currentNode: Node = frontier.popleft()

        if explored.get(currentNode.state, False):
            continue

        if problem.is_goal(currentNode.state):
            return currentNode
        
        frontier.extend(expand(problem, currentNode))

        explored[currentNode.state] = True

    return None

## Exploration en profondeur du graphe des états

### Question 5 : implémenter l'exploration en profondeur du graphe des états (voir slide 27 du cours)

In [222]:
def depth_first_graph_search(problem: Problem):
    frontier = []
    explored: dict[tuple, bool] = {}

    frontier.append(Node(problem.initial))

    while len(frontier) > 0:
        currentNode: Node = frontier.pop()

        if explored.get(currentNode.state, False):
            continue

        if problem.is_goal(currentNode.state):
            return currentNode
        
        frontier.extend(expand(problem, currentNode))

        explored[currentNode.state] = True

    return None

## Exploration gloutonne du graphe des états

### Question 6 : implémenter l'exploration gloutonne du graphe des états (voir slides 27 et 34 du cours)

In [223]:
def greedy_graph_search(problem: Problem, h):
    frontier: list[Node] = []
    explored: dict[tuple, bool] = {}

    frontier.append(Node(problem.initial))

    while len(frontier) > 0:
        currentNode: Node = frontier.pop()

        if explored.get(currentNode.state, False):
            continue

        if problem.is_goal(currentNode.state):
            return currentNode
        
        frontier.extend(expand(problem, currentNode))
        frontier.sort(key=h, reverse=True)

        explored[currentNode.state] = True

    return None

## Exploration A* du graphe des états

### Question 7 : implémenter l'exploration A* du graphe des états (voir slides 27 et 37 du cours)

In [224]:
def astar_graph_search(problem, h, g):
    def star(node: Node): return g(node)+h(node)

    frontier: list[Node] = []
    explored: dict[tuple, bool] = {}

    frontier.append(Node(problem.initial))

    while len(frontier) > 0:
        currentNode: Node = frontier.pop()

        if explored.get(currentNode.state, False):
            continue

        if problem.is_goal(currentNode.state):
            return currentNode
        
        frontier.extend(expand(problem, currentNode))
        frontier.sort(key=star, reverse=True)

        explored[currentNode.state] = True

    return None

# Définition d'un labyrinthe

On commence par définir la classe `Cell` (pour représenter les cases d'un labyrinthe) puis la classe `Labyrinth` pour représenter le labyrinthe constitué de cases.

In [225]:
from numpy.random import randint
import matplotlib.pyplot as plt


class Stack:
    """ définition d'une pile à l'aide de la classe List """
    def __init__(self, lst: list = []):
        self.lst = lst

    def empty(self):
        return self.lst == []

    def push(self, x):
        self.lst.append(x)

    def pop(self):
        if self.empty():
            raise ValueError("pile vide")
        return self.lst.pop()


class Cell:
    def __init__(self, N=False, E=False, S=False, W=False):
        self.N = N
        self.E = E
        self.S = S
        self.W = W
    

class Labyrinth:
    def __init__(self, p, q, tab=None):
        self.p = p
        self.q = q
        if tab == None:
            self.tab = [[Cell() for j in range(q)] for i in range(p)]
        else:
            self.tab = tab
        
    def entrance(self):
        return (0, 0)

    def exit(self):
        return (self.p-1, self.q-1)

    def create(self):
        pile = Stack()
        dejavu = [[False for j in range(self.q)] for i in range(self.p)]
        i, j = randint(self.p), randint(self.q)
        pile.push((i, j))
        dejavu[i][j] = True
        while not pile.empty():
            (i, j) = pile.pop()
            v = []
            if j < self.q-1 and not dejavu[i][j+1]:
                v.append('E')
            if i > 0 and not dejavu[i-1][j]:
                v.append('N')
            if j > 0 and not dejavu[i][j-1]:
                v.append('W')
            if i < self.p-1 and not dejavu[i+1][j]:
                v.append('S')
            if len(v) > 1:
                pile.push((i, j))
            if len(v) > 0:
                c = v[randint(len(v))]
                if c == 'N':
                    self.tab[i][j].N = True
                    self.tab[i-1][j].S = True
                    dejavu[i-1][j] = True
                    pile.push((i-1, j))
                elif c == 'W':
                    self.tab[i][j].W = True
                    self.tab[i][j-1].E = True
                    dejavu[i][j-1] = True
                    pile.push((i, j-1))
                elif c == 'S':
                    self.tab[i][j].S = True
                    self.tab[i+1][j].N = True
                    dejavu[i+1][j] = True
                    pile.push((i+1, j))
                else:
                    self.tab[i][j].E = True
                    self.tab[i][j+1].W = True
                    dejavu[i][j+1] = True
                    pile.push((i, j+1))

    def show(self, sol=Stack()):
        fig = plt.figure(1, figsize=(self.q/3.2, self.p/3.2))
        ax = fig.add_subplot(111)
        ax.axis([-1, self.q+1, -1, self.p+1])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.plot([0, 0, self.q, self.q, 0],
                 [0, self.p, self.p, 0, 0], linewidth=2)
        for i in range(self.p-1):
            for j in range(self.q):
                if not self.tab[i][j].S:
                    ax.plot([j, j+1], [self.p-i-1, self.p-i-1], 'b')
        for j in range(self.q-1):
            for i in range(self.p):
                if not self.tab[i][j].E:
                    ax.plot([j+1, j+1], [self.p-i, self.p-i-1], 'b')
        if not sol.empty():
            X, Y = [], []
            while not sol.empty():
                (i, j) = sol.pop()
                X.append(j+.5)
                Y.append(self.p-i-.5)
            ax.plot(X, Y, 'r', linewidth=2)
        plt.show()

## Test de la classe Labyrinth

In [ ]:
# création et affichage d'un labyrinthe généré aléatoirement à partir de la taille souhaitée
lab1 = Labyrinth(12, 18)
lab1.create()
lab1.show()

print("entrée : " + str(lab1.entrance()))
print("sortie : " + str(lab1.exit()))


# création et affichage d'un labyrinthe à partir d'un tableau de cellules, indiquant les murs autour de chaque case
tab = [
    [Cell(False, True, True, False), Cell(False, True, False, True), Cell(False, True, True, True), Cell(False, False, True, True)],
    [Cell(True, True, False, False), Cell(False, False, False, True), Cell(True, False, True, False), Cell(True, False, True, False)],
    [Cell(False, True, True, False), Cell(False, True, False, True), Cell(True, False, True, True), Cell(True, False, True, False)],
    [Cell(True, False, False, False), Cell(False, True, True, False), Cell(False, False, False, True), Cell(True, False, True, False)],
    [Cell(False, True, False, False), Cell(True, True, False, True), Cell(False, True, False, True), Cell(True, False, False, True)],
]

lab2 = Labyrinth(5, 4, tab)
lab2.show()
print(lab2.tab[0][0].N)

print("entrée : " + str(lab2.entrance()))
print("sortie : " + str(lab2.exit()))


# Résolution de labyrinthe

La résolution d'un problème de type labyrinthe implique de naivguer dans une grille 2D avec des cases pouvant contenir des murs dans les 4 directions autour de chaque case (nord, est, sud et ouest). 

Il est nécessaire d'avoir un attribut `labyrinth` pour représenter le labyrinthe.

Les états sont les positions des cellules sous la forme `(x, y)`, comme par exemple, `(4, 2)`. 

Les actions peuvent, par exemple, être représentées sous la forme de mouvements `(dx, dy)` d'une case à une autre, tel que `(0, -1)`, ce qui signifie de ne pas modifier la coordonnée `x` et de décrémenter la coordonnée `y` de 1 ou bien par des chaînes de caractères `H`, `B`, `G` et `D`, correspondant aux actions "haut", "bas", "gauche" et "droite".

### Question 8 : compléter les méthodes de la classe `LabyrinthProblem`, en reprenant celles de la classe `Problem` que vous souhaitez implémenter

In [227]:
class LabyrinthProblem(Problem):
    
    def __init__(self, labyrinth: Labyrinth):
        super().__init__(initial=labyrinth.entrance(), goal=labyrinth.exit())
        self.labyrinth = labyrinth
    
        
    def actions(self, state: tuple[int,int]):
        ret = []

        currentCell = self.labyrinth.tab[state[0]][state[1]]


        if currentCell.N: ret.append("H")
        if currentCell.S: ret.append("B")
        if currentCell.W: ret.append("G")
        if currentCell.E: ret.append("D")

        return ret
    
    
    def result(self, state: tuple[int,int], action: chr):
        newState: tuple[int, int]

        match action:
            case "H":
                newState = (state[0]-1, state[1])
            case "B":
                newState = (state[0]+1, state[1])
            case "G":
                newState = (state[0], state[1]-1)
            case "D":
                newState = (state[0], state[1]+1)
        
        return newState
    
    
    def h(self, node: Node) -> int:
        goal = self.labyrinth.exit()
        current: tuple[int, int] = node.state

        return abs(current[0]-goal[0]) + abs(current[1]-goal[1])

    def g(self, node: Node): 
        return node.path_cost

### Question 9 : tester la résolution de problèmes de labyrinthes, en utilisant la classe `LabyrinthProblem`, ainsi que les fonctions correspondant aux différents algorithmes de recherche + affichage du chemin parcouru, sur le labyrinthe

In [ ]:
labProb2 = LabyrinthProblem(labyrinth=lab2)
labProb2.labyrinth.show()

print("\nexploration en largeur (en arbre)")
res2bt = breadth_first_tree_search(labProb2)
etats2bt = path_states(res2bt)
print("\tétats sur le chemin vers la sortie : " + str(etats2bt))
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res2bt)))
labProb2.labyrinth.show(Stack(etats2bt))

# print("\nexploration en profondeur (en arbre)")
# res2dt = depth_first_tree_search(labProb2)
# etats2dt = path_states(res2dt)
# print("\tétats sur le chemin vers la sortie : " + str(etats2dt))
# print("\tactions sur le chemin vers la sortie : " + str(path_actions(res2dt)))
# labProb2.labyrinth.show(Stack(etats2dt))

print("\nexploration en largeur (en graphe)")
res2bg = breadth_first_graph_search(labProb2)
etats2bg = path_states(res2bg)
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res2bg)))
labProb2.labyrinth.show(Stack(etats2bg))

print("\nexploration en profondeur (en graphe)")
res2dg = depth_first_graph_search(labProb2)
etats2dg = path_states(res2dg)
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res2dg)))
labProb2.labyrinth.show(Stack(etats2dg))

print("\nexploration gloutonne")
res2gl = greedy_graph_search(labProb2, labProb2.h)
etats2gl = path_states(res2gl)
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res2gl)))
labProb2.labyrinth.show(Stack(etats2gl))

print("\nexploration A*")
res2as = astar_graph_search(labProb2, labProb2.h, labProb2.g)
etats2as = path_states(res2as)
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res2as)))
labProb2.labyrinth.show(Stack(etats2as))

labProb1 = LabyrinthProblem(labyrinth=lab1)
labProb1.labyrinth.show()

print("\nexploration gloutonne")
res1gl = greedy_graph_search(labProb1, labProb1.h)
etats1gl = path_states(res1gl)
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res1gl)))
labProb1.labyrinth.show(Stack(etats1gl))

print("\nexploration A*")
res1as = astar_graph_search(labProb1, labProb1.h, labProb1.g)
etats1as = path_states(res1as)
print("\tactions sur le chemin vers la sortie : " + str(path_actions(res1as)))
labProb1.labyrinth.show(Stack(etats1as))

labTest = Labyrinth(50,50)
labTest.create()
labProbTest = LabyrinthProblem(labyrinth=labTest)
labProbTest.labyrinth.show()

print("\nexploration gloutonne")
%timeit greedy_graph_search(labProbTest, labProbTest.h)

print("\nexploration A*")
%timeit astar_graph_search(labProbTest, labProbTest.h, labProbTest.g)

### Question ? : taquin

In [ ]:
from numpy.random import randint
import matplotlib.pyplot as plt

class Puzzle15:
    def __init__(self, tiles, width, height):
        self.dimensions = (width, height)
        self.tiles = tiles
        self.emptyIndex = self.tiles.index(0)
        self.solution = [self.emptyIndex]

    def surroundingPieces(self, z):
        x = z%self.dimensions[0]
        y = math.floor(z/self.dimensions[1])

        left  = None if x == 0 else z - 1
        up    = None if y == 0 else z - self.dimensions[0]
        right = None if x == (self.dimensions[0]-1) else z + 1
        down  = None if y == (self.dimensions[1]-1) else z + self.dimensions[0]

        return list(filter(None, [left, up, right, down]))

    def shuffle(self, moves):
        lastPiece = self.emptyIndex #init lastPiece

        for _ in range(moves):
            #get surrounding pieces
            pieces = self.surroundingPieces(self.emptyIndex)

            #remove the last piece we moved
            if lastPiece in pieces:
                pieces.remove(lastPiece)

            #select a piece
            pieceIndex = random.choice(pieces)
            self.solution.insert(0, pieceIndex) #insert move in solution

            #swap        
            self.tiles[empty] = self.tiles[pieceIndex]
            lastPiece = empty  #store this piece index

            self.tiles[pieceIndex] = 0
            empty = pieceIndex #store new empty index

    def show(self):
        print(self.tiles)

class Puzzle15Problem(Problem):
    def __init__(self, puzzle: Puzzle15):
        super().__init__(initial=puzzle.emptyTile(), goal=puzzle.goal())
        self.puzzle = puzzle
    
    def actions(self, emptyPos: int):
        ret = []

        if emptyPos%self.puzzle.dimensions[1]: ret.append("H")
        if emptyPos%self.puzzle.dimensions[1]: ret.append("B")
        if emptyPos%self.puzzle.dimensions[0]: ret.append("G")
        if emptyPos%self.puzzle.dimensions[0]: ret.append("D")

        return ret
    
    
    def result(self, state: tuple[int,int], action: chr):
        newState: tuple[int, int]

        match action:
            case "H":
                newState = (state[0]-1, state[1])
            case "B":
                newState = (state[0]+1, state[1])
            case "G":
                newState = (state[0], state[1]-1)
            case "D":
                newState = (state[0], state[1]+1)
        
        return newState
    
    
    def h(self, node: Node) -> int:
        goal = self.labyrinth.exit()
        current: tuple[int, int] = node.state

        return abs(current[0]-goal[0]) + abs(current[1]-goal[1])

    def g(self, node: Node): 
        return node.path_cost

### Question 10 : comparer le nombre de noeuds générés et visités, par algorithme

## Pour lab1

Glouton :
- Noeuds generes :
- Noeuds visites :

A* :
- Noeuds generes :
- Noeuds visites :

## Pour lab2

Largeur :
- Noeuds generes :
- Noeuds visites :

Profondeur :
- Noeuds generes :
- Noeuds visites :

Glouton :
- Noeuds generes :
- Noeuds visites :

A* :
- Noeuds generes :
- Noeuds visites :